In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder

df_candidatos = pd.read_csv("candidatos.csv")
df_candidatos


#dime que librerias debo importar para construir un modelo de clasificación



,vacante_id,educacion,experiencia_anios,experiencia_sector,certificaciones,puntaje_test,puntaje_entrevista,nivel_ingles,referencia_interna,contratado
0,Vacante_1,Secundario,13,RRHH,0,4.94,2.31,Básico,0,0
1,Vacante_1,Secundario,15,Marketing,3,4.00,4.03,Básico,0,0
2,Vacante_1,Postgrado,11,Finanzas,4,5.71,7.98,Intermedio,0,0
3,Vacante_1,Postgrado,13,RRHH,0,1.42,9.77,Intermedio,1,1
4,Vacante_1,Universitario,3,Marketing,1,9.83,7.68,Básico,1,0
...,...,...,...,...,...,...,...,...,...,...
92,Vacante_10,Universitario,7,RRHH,1,8.94,9.67,Avanzado,0,0
93,Vacante_10,Postgrado,8,RRHH,5,9.79,1.51,Avanzado,0,0
94,Vacante_10,Terciario,1,RRHH,2,5.20,5.22,Avanzado,0,0
95,Vacante_10,Universitario,15,Marketing,1,3.34,9.42,Avanzado,0,0


In [ ]:
categorical_cols = ["educacion", "experiencia_sector", "nivel_ingles"]
numerical_cols = ["experiencia_anios", "certificaciones", "puntaje_test", "puntaje_entrevista", "referencia_interna"]
target = "contratado"

#codifico las variables categóricas
encoder = OneHotEncoder(sparse_output=False)
encoder_cats = encoder.fit_transform(df_candidatos[categorical_cols])
encoded_cat_df = pd.DataFrame(encoder_cats, columns=encoder.get_feature_names_out(categorical_cols))

#combino con las numericas
X = pd.concat([encoded_cat_df, df_candidatos[numerical_cols].reset_index(drop=True)], axis=1)
y = df_candidatos[target]


In [ ]:

#divido los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42)

#entreno el modelo
modelo = RandomForestClassifier(n_estimators=100, random_state=42)
modelo.fit(X_train, y_train)



RandomForestClassifier(random_state=42)

In [ ]:
#evaluo el modelo
y_pred = modelo.predict(X_test)
y_proba = modelo.predict_proba(X_test)[:,1]


In [ ]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
y_test

,contratado
62,1
40,0
93,0
18,0
81,0
83,1
64,0
42,0
10,0
0,0


In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.94      0.89        17
           1       0.00      0.00      0.00         3

    accuracy                           0.80        20
   macro avg       0.42      0.47      0.44        20
weighted avg       0.72      0.80      0.76        20



In [ ]:
X_all = pd.concat([encoded_cat_df, df_candidatos[numerical_cols].reset_index(drop=True)], axis=1)
df_candidatos["probabilidad_contratacion"] = modelo.predict_proba(X_all)[:, 1]

In [ ]:
df_candidatos

,vacante_id,educacion,experiencia_anios,experiencia_sector,certificaciones,puntaje_test,puntaje_entrevista,nivel_ingles,referencia_interna,contratado,probabilidad_contratacion
0,Vacante_1,Secundario,13,RRHH,0,4.94,2.31,Básico,0,0,0.40
1,Vacante_1,Secundario,15,Marketing,3,4.00,4.03,Básico,0,0,0.10
2,Vacante_1,Postgrado,11,Finanzas,4,5.71,7.98,Intermedio,0,0,0.18
3,Vacante_1,Postgrado,13,RRHH,0,1.42,9.77,Intermedio,1,1,0.79
4,Vacante_1,Universitario,3,Marketing,1,9.83,7.68,Básico,1,0,0.16
...,...,...,...,...,...,...,...,...,...,...,...
92,Vacante_10,Universitario,7,RRHH,1,8.94,9.67,Avanzado,0,0,0.13
93,Vacante_10,Postgrado,8,RRHH,5,9.79,1.51,Avanzado,0,0,0.45
94,Vacante_10,Terciario,1,RRHH,2,5.20,5.22,Avanzado,0,0,0.12
95,Vacante_10,Universitario,15,Marketing,1,3.34,9.42,Avanzado,0,0,0.11


In [ ]:
# Crear gráficos interactivos para poder visualizar los candidatos con mayor probabilidad de contratación

import pandas as pd
import plotly.graph_objects as go

# Asume que ya tienes tu DataFrame cargado como df_candidatos
df = df_candidatos.copy()

# Ordena dentro de cada vacante por probabilidad descendente
df_sorted = df.sort_values(['vacante_id', 'probabilidad_contratacion'], ascending=[True, False])

# Lista de vacantes únicas
vacantes = df_sorted['vacante_id'].unique().tolist()

# Crea la figura con un trace por vacante (inicialmente solo visible la primera)
fig = go.Figure()

for i, vac in enumerate(vacantes):
    sub = df_sorted[df_sorted['vacante_id'] == vac]
    fig.add_trace(
        go.Bar(
            x=sub.index.astype(str),
            y=sub['probabilidad_contratacion'],
            name=f"Vacante {vac}",
            visible=(i == 0),
            hovertemplate=
                "<b>Candidato</b>: %{x}<br>" +
                "<b>Probabilidad</b>: %{y:.2%}<br>" +
                "<b>Educación</b>: %{customdata[0]}<br>" +
                "<b>Años exp.</b>: %{customdata[1]}<br>" +
                "<b>Sector exp.</b>: %{customdata[2]}<br>" +
                "<extra></extra>",
            customdata=sub[['educacion','experiencia_anios','experiencia_sector']].values
        )
    )

# Botones de dropdown para cambiar de vacante
buttons = [
    dict(
        label=str(vac),
        method="update",
        args=[
            {"visible": [v == vac for v in vacantes]},
            {"title": f"Vacante {vac}"}
        ]
    )
    for vac in vacantes
]

fig.update_layout(
    updatemenus=[dict(
        active=0,
        buttons=buttons,
        x=0, y=1.15,
        xanchor='left', yanchor='top'
    )],
    title=f"Vacante {vacantes[0]}",
    xaxis_title="Índice de Candidato",
    yaxis_title="Probabilidad de Contratación",
    showlegend=False,
    margin=dict(t=100)
)

fig.show()



In [ ]:
# Método 1: usando joblib (recomendado para modelos de scikit-learn)
import joblib

# Guardar
joblib.dump(modelo, 'modelo_entrenado.pkl')

# Cargar
model = joblib.load('modelo_entrenado.pkl')